## Урок 6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [19]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp

from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno


# plt.style.use('fivethirtyeight')
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score, \
                            classification_report, plot_confusion_matrix, plot_precision_recall_curve, \
                            precision_recall_curve, recall_score, plot_roc_curve)

import xgboost as xgb
import catboost as cb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset.
Data Set Information:

This has been col-
lected using direct questionnaires from the patients of Sylhet Diabetes
Hospital in Sylhet, Bangladesh and approved by a doctor.

In [5]:
data_df = pd.read_csv('diabetes_data_upload.csv')
data_df

# train = reduce_mem_usage(pd.read_csv(r"./competitive-data-analysis_coursework/train.csv"))
# test = reduce_mem_usage(pd.read_csv(r"./competitive-data-analysis_coursework/test.csv"))
# client_profile = reduce_mem_usage(pd.read_csv(r"./competitive-data-analysis_coursework/client_profile.csv"))

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


In [6]:
list_replace = {'Positive': '1', 'Negative': '0'}
data_df['class'] = data_df['class'].map(list_replace)
data_df['class'] = data_df['class'].astype('int')

In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   sudden weight loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital thrush      520 non-null    object
 8   visual blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed healing     520 non-null    object
 12  partial paresis     520 non-null    object
 13  muscle stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [11]:
df = pd.get_dummies(data_df)

In [14]:
df = df.rename(columns={'class': 'target'})

In [15]:
df

,Age,target,Gender_Female,Gender_Male,Polyuria_No,Polyuria_Yes,Polydipsia_No,Polydipsia_Yes,sudden weight loss_No,sudden weight loss_Yes,...,delayed healing_No,delayed healing_Yes,partial paresis_No,partial paresis_Yes,muscle stiffness_No,muscle stiffness_Yes,Alopecia_No,Alopecia_Yes,Obesity_No,Obesity_Yes
0,40,1,0,1,1,0,0,1,1,0,...,0,1,1,0,0,1,0,1,0,1
1,58,1,0,1,1,0,1,0,1,0,...,1,0,0,1,1,0,0,1,1,0
2,41,1,0,1,0,1,1,0,1,0,...,0,1,1,0,0,1,0,1,1,0
3,45,1,0,1,1,0,1,0,0,1,...,0,1,1,0,1,0,1,0,1,0
4,60,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,1,1,0,0,1,0,1,0,1,...,0,1,0,1,1,0,1,0,1,0
516,48,1,1,0,0,1,0,1,0,1,...,0,1,0,1,1,0,1,0,1,0
517,58,1,1,0,0,1,0,1,0,1,...,1,0,0,1,0,1,1,0,0,1
518,32,0,1,0,1,0,1,0,1,0,...,0,1,1,0,1,0,0,1,1,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns='target'), df['target'], test_size=0.2, random_state=99)

In [17]:
model_xgb = XGBClassifier(random_state=99)
model_xgb.fit(X_train, y_train)
y_proba = model_xgb.predict_proba(X_test)[:, 1]
y_pred = model_xgb.predict(X_test)

[15:44:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [20]:
result = {'metrics': ['roc_auc', 'f1score', 'precision', 'recall']}

def metrics(y_test, y_pred, y_proba):
    return [roc_auc_score(y_test, y_proba),
            f1_score(y_test, y_pred),
            precision_score(y_test, y_pred),
            recall_score(y_test, y_pred)]

result['XGBClassifier_base'] = metrics(y_test, y_pred, y_proba)

In [21]:
result

{'metrics': ['roc_auc', 'f1score', 'precision', 'recall'],
 'XGBClassifier_base': [0.9798305768455021,
  0.9538461538461538,
  0.9841269841269841,
  0.9253731343283582]}

PU-learning. (random negative sampling)

In [22]:
for k in [10, 20, 30, 40, 50]:

    pos_ind = df.loc[df.target == 1].index
    n_samples = round(k*len(pos_ind)/100)
    pos_samples = np.random.choice(pos_ind, size=n_samples, replace=False)
    data_rs = df.copy()
    data_rs['PU_labels'] = 0
    data_rs.loc[pos_samples, 'PU_labels'] = 1
    neg_ind = data_rs.loc[data_rs['PU_labels'] == 0].index
    neg_samples = np.random.choice(neg_ind, size=n_samples, replace=False)
    rs_train = pd.concat([data_rs.loc[pos_samples], data_rs.loc[neg_samples]])
    rs_test = data_rs.drop(index=(np.append(pos_samples, neg_samples)))
    

    model_xgb_rs = XGBClassifier(random_state=13)
    model_xgb_rs.fit(rs_train.drop(
        columns=['target', 'PU_labels']), rs_train['PU_labels'])
    y_proba = model_xgb_rs.predict_proba(
        rs_test.drop(columns=['target', 'PU_labels']))[:, 1]
    y_pred = model_xgb_rs.predict(
        rs_test.drop(columns=['target', 'PU_labels']))
    
    result['XGBClassifier_RS_' +
           str(k)] = metrics(rs_test['target'], y_pred, y_proba)

[15:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [23]:
pd.DataFrame(result).set_index('metrics')

,XGBClassifier_base,XGBClassifier_RS_10,XGBClassifier_RS_20,XGBClassifier_RS_30,XGBClassifier_RS_40,XGBClassifier_RS_50
metrics,,,,,,
roc_auc,0.979831,0.864057,0.943089,0.905400,0.961240,0.961330
f1score,0.953846,0.703786,0.758427,0.717391,0.770642,0.748201
precision,0.984127,0.887640,0.992647,0.925234,0.943820,0.962963
recall,0.925373,0.583026,0.613636,0.585799,0.651163,0.611765
